In [1]:
import pandas as pd

In [2]:
gap_train= pd.read_pickle('./temp_result/train_kaggle_processed')
gap_test= pd.read_pickle('./temp_result/test_kaggle_processed')
gap_valid= pd.read_pickle('./temp_result/valid_kaggle_processed')

In [3]:
def label(A,B):
    if A is True:
        return 0
    if B is True:
        return 1
    return 2

In [4]:
gap_train.tail()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,...,B_dist,A_pos,B_pos,pron_pos,A_idx,B_idx,pron_idx,A_vector,B_vector,pron_vector
1995,test-1996,"The sole exception was Wimbledon, where she pl...",She,479,Goolagong Cawley,400,True,Peggy Michel,432,False,...,0.020,0.220779,0.233766,0.259740,"[102, 103, 104, 105, 106]","[110, 111, 112, 113, 114]",[123],"[[-0.3187313, 0.46942553, 0.541859, 0.0963608,...","[[-0.37300664, -0.315253, 0.040081125, 0.53844...","[[-0.7649605, -0.4366843, 0.38759252, 0.534622..."
1996,test-1997,"According to news reports, both Moore and Fily...",her,338,Esther Sheryl Wood,263,True,Barbara Morgan,404,False,...,-0.028,0.504762,0.790476,0.657143,"[62, 63, 64, 65, 66]","[96, 97, 98, 99, 100]",[80],"[[-0.211019, -0.036700837, -0.027644457, -0.34...","[[-0.2204119, 0.5043704, -0.4220593, 0.2550880...","[[-0.14545152, -0.01811517, 0.8394235, -0.9302..."
1997,test-1998,"In June 2009, due to the popularity of the Sab...",She,328,Kayla,364,True,Natasha Henstridge,412,False,...,-0.034,0.461078,0.520958,0.419162,"[97, 98]","[111, 112, 113, 114, 115, 116]",[90],"[[0.20733312, 0.7342792, 0.03844169, 0.3875529...","[[-0.21739139, 0.05455912, 0.1606792, 0.440639...","[[-0.8260292, -0.59548515, 0.04561778, -0.1782..."
1998,test-1999,She was delivered to the Norwegian passenger s...,she,305,Irma,255,True,Bergen,274,False,...,0.008,0.420561,0.457944,0.495327,"[67, 68]","[72, 73]",[80],"[[-0.805637, -0.5994673, -0.5566493, 0.0688710...","[[0.1469631, 0.8217377, 0.33704713, 0.3077713,...","[[-0.97279143, 0.2389887, -0.22996032, -0.0032..."
1999,test-2000,"Meg and Vicky each have three siblings, and ha...",her,275,Vicky Austin,217,True,Polly O'Keefe,260,False,...,0.006,0.300000,0.361538,0.384615,"[48, 49, 50, 51, 52, 53]","[60, 61, 62, 63, 64, 65, 66]",[68],"[[-0.2613115, -0.83725923, -0.47856662, 0.4980...","[[-0.20457774, 0.9843961, -0.32598388, 0.59570...","[[-0.56508684, 0.04315004, -0.17839415, -0.330..."


In [5]:
import torch
import torch.nn as nn

In [6]:
gap_train["seq_length"] = gap_train.vector.map(lambda x: x.shape[0])
gap_valid["seq_length"] = gap_valid.vector.map(lambda x: x.shape[0])
gap_test["seq_length"] = gap_test.vector.map(lambda x: x.shape[0])

In [7]:
gap_train = gap_train.sort_values(by=['seq_length'], ascending=False)
gap_valid = gap_train.sort_values(by=['seq_length'], ascending=False)
gap_test = gap_train.sort_values(by=['seq_length'], ascending=False)

In [8]:
gap_train = gap_train.reset_index(drop = True)
gap_valid = gap_valid.reset_index(drop = True)
gap_test = gap_test.reset_index(drop = True)

In [9]:
gap_train["label"] = gap_train.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
gap_test["label"] = gap_test.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
gap_valid["label"] = gap_valid.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)

In [10]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        #print ("scores")
        #print (scores.shape)
        #print ("scores")
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [11]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
            #print ("inside")
            #print (mask.size())
            #print ("inside")
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

In [12]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [13]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [14]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        #print (x.size())
        #print (mask.size())
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [15]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [16]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [17]:
import copy
c = copy.deepcopy

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

attn = MultiHeadedAttention(h = 8, d_model = 1024)
ff = PositionwiseFeedForward(d_model = 1024, d_ff = 1024, dropout = 0.5)
enc = Encoder(EncoderLayer(size = 1024, self_attn = c(attn),feed_forward = c(ff), dropout = 0.5), N = 2)

In [18]:
import numpy as np
import math
import torch as F

In [19]:
def get_batch(df,start,batch_size,dim = 1024):
    num = df.shape[0]
    dim = df.vector.iloc[0].shape[1]
    #print (dim)
    end = min(num,start + batch_size)
    batch_data = df.iloc[start:end,:]
    max_seq_length = df.loc[start,"seq_length"]
    if max_seq_length<32:
        max_seq_length = 32
    padded = np.zeros((end-start,max_seq_length,dim))
    sentence_vector = list(batch_data.vector)
    for i,v in enumerate(sentence_vector):
        padded[i,0:v.shape[0],:] = v   
    label = batch_data.label
    A_mask = np.zeros((end-start,max_seq_length,max_seq_length))
    B_mask = np.zeros((end-start,max_seq_length,max_seq_length))
    pron_mask = np.zeros((end-start,max_seq_length,max_seq_length))
    for i in range(0,end-start):
        A_mask[i,df.A_idx.loc[start+i],df.A_idx.loc[start+i]] = 0.001      
        B_mask[i,df.B_idx.loc[start+i],df.B_idx.loc[start+i]] = 0.001
        pron_mask[i,df.pron_idx.iloc[start+i],df.pron_idx.iloc[start+i]] = 0.001
    A_mask = torch.tensor(A_mask).unsqueeze(-3)
    B_mask = torch.tensor(B_mask).unsqueeze(-3)
    pron_mask = torch.tensor(pron_mask).unsqueeze(-3)
    return torch.Tensor(padded),torch.LongTensor(np.array(label)),torch.cat((A_mask,B_mask,pron_mask),dim = 1)

In [20]:
import torchvision.models as models
class CNN(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, enc):
        super(CNN, self).__init__()
        self.enc = enc
        self.cnn = nn.Sequential(
  nn.Conv2d(19, 64, kernel_size=(5, 5), stride=(4, 4), padding=(2, 2)),
  nn.ReLU(),
  nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
  nn.Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2)),
  nn.ReLU(),
  nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
  nn.Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  nn.ReLU(),
  nn.Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  nn.ReLU(),
  nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  nn.ReLU(),
  nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
  nn.Conv2d(256, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  nn.AdaptiveAvgPool2d(output_size = (1,1))
)
    def forward(self, padded,mask_feature):
        mask = (padded.mean(dim = -1) != 0).unsqueeze(-2)
        self.out = enc(padded,mask)
        cnn_input = torch.cat((enc.layers[0].self_attn.attn,enc.layers[1].self_attn.attn,mask_feature.float()),dim = 1)
        cnn_out = self.cnn(cnn_input)
        return cnn_out

In [21]:
cnn_model = CNN(enc)
# This was important from their code. 
# Initialize parameters with Glorot / fan_avg.
for p in cnn_model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
cnn_model.cuda()
None

In [22]:
loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')
loss_fn.cuda()
optimizer = torch.optim.Adam(cnn_model.parameters(), lr = 0.005)

In [23]:
EPOCH = 20
BATCH_SIZE = 8
start = 0
data_num = 2000
for e in range(EPOCH):
    start = 0
    while start <= data_num-1:
        padded, label, mask_feature = get_batch(gap_train,start,BATCH_SIZE)
        padded = padded.cuda()
        label = label.cuda()
        cnn_model.train()
        mask_feature = mask_feature.cuda()
        start = start + BATCH_SIZE
        cnn_out = cnn_model(padded,mask_feature)
        cnn_out = cnn_out.view(-1,3)
        #print (label.size())
        cnn_out = torch.nn.Softmax(dim = -1)(cnn_out)
        loss = loss_fn(cnn_out,label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print ("loss at the end of epoch "+str(e))
    print (loss.item())

    

loss at the end of epoch 0
1.05144464969635
loss at the end of epoch 1
1.05144464969635
loss at the end of epoch 2
1.05144464969635
loss at the end of epoch 3
1.05144464969635
loss at the end of epoch 4
1.05144464969635
loss at the end of epoch 5
1.05144464969635
loss at the end of epoch 6
1.05144464969635
loss at the end of epoch 7
1.05144464969635
loss at the end of epoch 8
1.05144464969635
loss at the end of epoch 9
1.05144464969635
loss at the end of epoch 10
1.05144464969635
loss at the end of epoch 11
1.05144464969635
loss at the end of epoch 12
1.05144464969635
loss at the end of epoch 13
1.05144464969635
loss at the end of epoch 14
1.05144464969635
loss at the end of epoch 15
1.05144464969635


KeyboardInterrupt: 